# ya_cv_hw6_b

In [ ]:
!mkdir temp
!unzip dataset.zip -d temp

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os

# Загружаем предобученную модель ResNet
model = models.resnet50(pretrained=True)
model.eval()  # Переводим модель в режим оценки

# Преобразования для входных изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def extract_features(image_path):
    # Открываем изображение
    img = Image.open(image_path).convert('RGB')
    img = transform(img).unsqueeze(0)  # Применяем преобразования и добавляем batch размер

    # Получаем фичи из модели
    with torch.no_grad():
        features = model(img)

    return features.squeeze().numpy()

# Пример извлечения признаков для одного изображения
features = extract_features('/content/temp/dataset/0001.png')


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

image_dir = '/content/temp/dataset'
image_files = os.listdir(image_dir)

# Получаем признаки для всех изображений
features_list = []
image_paths = []

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    features = extract_features(image_path)
    features_list.append(features)
    image_paths.append(image_file)

# Преобразуем в numpy массив
features_array = np.array(features_list)

# Вычисляем косинусное сходство
similarity_matrix = cosine_similarity(features_array)

# Функция для нахождения 6 наиболее похожих изображений
def get_similar_images(query_idx, top_k=6):
    # Сортируем сходства по убыванию, исключая само изображение
    similarities = similarity_matrix[query_idx]
    sorted_idx = np.argsort(similarities)[::-1][1:top_k+1]

    # Возвращаем имена файлов
    return [image_paths[i] for i in sorted_idx]

# Пример: для первого изображения находим 6 похожих
similar_images = get_similar_images(0)
print(similar_images)


['6611.png', '7535.png', '0728.png', '3518.png', '3913.png', '2349.png']


In [ ]:
import csv

# Создаём список с результатами
results = []

for i, image_file in enumerate(image_paths):
    similar_images = get_similar_images(i)
    results.append([image_file, ' '.join(similar_images)])

# Сохраняем в CSV
output_file = '/content/submission.csv'
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['filename', 'ranking'])
    writer.writerows(results)

print(f"Результаты сохранены в {output_file}")


Результаты сохранены в /content/submission.csv


In [ ]:
# CLIP
!pip install open_clip_torch

import open_clip
import torch
from PIL import Image
from torchvision import transforms

import os

image_dir = '/content/temp/dataset'
image_files = os.listdir(image_dir)

# Загружаем CLIP модель
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model = model.eval().cuda()  # Если у тебя доступен GPU в Colab

tokenizer = open_clip.get_tokenizer('ViT-B-32')

def extract_features_clip(image_path):
    img = Image.open(image_path).convert('RGB')
    img = preprocess(img).unsqueeze(0).cuda()
    with torch.no_grad():
        image_features = model.encode_image(img)
    return image_features.cpu().squeeze(0).numpy()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/open_clip/factory.py:388: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


In [ ]:
from torch.utils.data import DataLoader, Dataset

class ImageDataset(Dataset):
    def __init__(self, file_paths, transform):
        self.file_paths = file_paths
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx]).convert('RGB')
        img = self.transform(img)
        return img, self.file_paths[idx]

# Создаём датасет и дата-лоадер
dataset = ImageDataset([os.path.join(image_dir, fname) for fname in image_files], preprocess)
loader = DataLoader(dataset, batch_size=64, shuffle=False)

features_list = []
names_list = []

for imgs, paths in loader:
    imgs = imgs.cuda()
    with torch.no_grad():
        image_features = model.encode_image(imgs)
    features_list.append(image_features.cpu())
    names_list.extend(paths)

# Объединяем
features_array = torch.cat(features_list, dim=0).numpy()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(features_array)


In [ ]:
import csv
import numpy as np

def get_similar_images(query_idx, top_k=6):
    similarities = similarity_matrix[query_idx]
    sorted_idx = np.argsort(similarities)[::-1][1:top_k+1]
    return [os.path.basename(names_list[i]) for i in sorted_idx]

# Запись
output_file = '/content/submission.csv'
with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['filename', 'ranking'])
    for i, path in enumerate(names_list):
        ranking = ' '.join(get_similar_images(i))
        writer.writerow([os.path.basename(path), ranking])
